# Prepare trainig data by combining historical data and new data

## Load recently obtained data

In [37]:
import numpy as np 
import pandas as pd 

path_ls = []
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        print(path)
        path_ls.append(path)

/kaggle/input/2025-03-27-6/2025-03-28 06_16_16.546768_historical.csv
/kaggle/input/new-sentiments/2025-03-19.csv
/kaggle/input/new-sentiments/2025-03-25.csv
/kaggle/input/new-sentiments/2025-03-21.csv
/kaggle/input/new-sentiments/2025-03-27.csv
/kaggle/input/new-sentiments/2025-03-17.csv


In [38]:
import warnings
warnings.filterwarnings("ignore")

In [39]:
path_ls = path_ls[1:-1]
dates = [p[-14:-4] for p in path_ls]
dates

['2025-03-19', '2025-03-25', '2025-03-21', '2025-03-27']

In [40]:
dates.sort()
print(dates)

['2025-03-19', '2025-03-21', '2025-03-25', '2025-03-27']


## Combine daily sentiment data

In [ ]:
base_path = "/input_path/2025-03-17.csv"
base_df = pd.read_csv(base_path)
def combine_sentiment(base_df, dates):
    for date in dates:
        path = "/input_path/" + date + ".csv"
        new_df = pd.read_csv(path)
        entries_to_add = new_df[new_df['Date'] >  base_df['Date'].max()]
        base_df = pd.concat([base_df, entries_to_add], ignore_index=True)

    return base_df
        
new_sentiments_df = combine_sentiment(base_df, dates)
new_sentiments_df = new_sentiments_df[['Date', 'Symbol', 'sentiment']]
new_sentiments_df = new_sentiments_df.dropna()
new_sentiments_df.head()

,Date,Symbol,sentiment
37,2025-03-12,ABBV,0.0
38,2025-03-13,ABBV,0.0
39,2025-03-14,ABBV,0.0
59,2025-03-14,ABT,-0.5
77,2025-03-12,ACN,-0.5


## Merge with historic data

In [42]:
hist_path = '/kaggle/input/2025-03-27-6/2025-03-28 06_16_16.546768_historical.csv'
historic_df = pd.read_csv(hist_path)
historic_df.tail(5)

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,month,...,sentiment,RSI,MA_20,MA_50,MA_200,log_return,RV_20,RV_50,symbol,time_idx
272149,272149,2025-03-21,115.500000,115.989998,114.349998,115.500000,41102800,0.0,0.0,3,...,NaN,69.664613,110.709501,109.526608,112.497300,-0.003457,0.256330,0.239104,XOM,2511
272150,272150,2025-03-24,115.680000,116.910004,115.580002,115.800003,14207500,0.0,0.0,3,...,NaN,70.900797,110.936001,109.722967,112.526918,0.002594,0.256078,0.235424,XOM,2512
272151,272151,2025-03-25,116.430000,117.639999,115.959999,116.589996,12081800,0.0,0.0,3,...,NaN,80.216766,111.279001,109.942858,112.558292,0.006799,0.249460,0.235355,XOM,2513
272152,272152,2025-03-26,117.680000,119.180000,117.680000,118.269997,14112800,0.0,0.0,3,...,NaN,79.665701,111.719501,110.141835,112.593921,0.014307,0.251652,0.230910,XOM,2514
272153,272153,2025-03-27,118.099998,119.070000,117.260002,117.889999,12734100,0.0,0.0,3,...,NaN,76.196086,112.106501,110.324690,112.633599,-0.003218,0.252702,0.231122,XOM,2515


In [43]:
historic_df.columns

Index(['Unnamed: 0', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume',
       'Dividends', 'Stock Splits', 'month', 'day', 'day_of_week', 'NASDAQ',
       'SNP', 'DJI', 'RUT', 'VIX', 'XLK', 'XLE', 'XLF', 'XLV', 'sentiment',
       'RSI', 'MA_20', 'MA_50', 'MA_200', 'log_return', 'RV_20', 'RV_50',
       'symbol', 'time_idx'],
      dtype='object')

In [44]:
def combine_historic_and_new_data(historic_df, new_df):

    # Merge A with B on Date and symbol
    merged = pd.merge(
        historic_df,
        new_df,
        on=['Date', 'symbol'],
        how='left',
        suffixes=('_A', '_B')
    )

    # Choose sentiment: take sentiment_A unless it's missing, then use sentiment_B
    merged['sentiment'] = merged['sentiment_A'].combine_first(merged['sentiment_B'])
    return merged

In [45]:
combined_df = combine_historic_and_new_data(historic_df, new_sentiments_df.rename(columns={'Symbol': 'symbol'}))

In [46]:
combined_df.sample(20)

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,month,...,MA_20,MA_50,MA_200,log_return,RV_20,RV_50,symbol,time_idx,sentiment_B,sentiment
111645,111645,2021-03-04,52.908170,54.374098,52.680698,53.270439,15847200,0.0,0.0,3,...,54.647907,53.409452,55.107781,0.007620,0.245043,0.270515,GILD,1494,NaN,NaN
26442,26442,2020-09-15,156.807999,158.751007,155.445999,157.806503,80430000,0.0,0.0,9,...,165.414223,159.255148,124.129830,0.016987,0.385770,0.375195,AMZN,1377,NaN,NaN
194891,194891,2018-11-21,3.830203,3.847047,3.557466,3.584715,1025496000,0.0,0.0,11,...,4.803533,5.765453,6.033791,-0.029751,1.074092,0.788086,NVDA,922,NaN,-0.200000
181305,181305,2018-12-24,57.313980,57.606355,56.018032,56.223488,11565833,0.0,0.0,12,...,59.491340,59.724317,55.440907,-0.024298,0.172480,0.150961,MRK,943,NaN,0.000000
153685,153685,2022-07-12,322.146089,324.088718,315.664105,315.888641,2058200,0.0,0.0,7,...,306.704544,298.019590,263.365708,-0.022852,0.279580,0.308097,LLY,1835,NaN,NaN
251339,251339,2021-10-05,189.439116,195.755905,189.439116,193.908890,4177300,0.0,0.0,10,...,187.954119,197.010518,197.516079,0.021470,0.250899,0.182007,UNP,1643,NaN,NaN
90010,90010,2018-03-28,95.583443,96.140615,94.286584,94.661232,9099700,0.0,0.0,3,...,98.345754,101.251201,99.494550,-0.008287,0.168346,0.230616,DIS,756,NaN,NaN
99239,99239,2024-01-25,93.073296,94.099512,92.369601,94.060417,2344500,0.0,0.0,1,...,93.083068,90.413223,88.174475,0.019410,0.166959,0.185595,EMR,2222,NaN,NaN
151448,151448,2016-04-01,61.075152,62.866461,61.015442,62.627621,4083600,0.0,0.0,4,...,60.912247,61.854988,67.668414,0.019391,0.285438,0.232156,LLY,255,NaN,NaN
237309,237309,2021-07-30,529.933310,538.265350,526.455827,535.005859,1412800,0.0,0.0,7,...,517.146840,487.596917,473.776197,0.011623,0.142494,0.220356,TMO,1597,NaN,NaN


## Impute Missing Sentiments with ARIMA model

In [47]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Train an ARIMA model to predict missing values
def impute_arima(df, column_name):
    # Make sure your column does not have NaNs initially
    series = df[column_name]

    # Fit ARIMA model to the non-missing data
    model = ARIMA(series, order=(5, 1, 0))  # You can experiment with the order
    model_fit = model.fit()

    # Predict the missing values using the fitted ARIMA model
    predictions = model_fit.predict(start=0, end=len(series)-1, typ='levels')

    # Fill NaN values with predicted values
    df[column_name].fillna(pd.Series(predictions, index=df.index), inplace=True)

    return df

In [48]:
combined_df = impute_arima(combined_df, 'sentiment')

In [49]:
combined_df.sample(100)

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,month,...,MA_20,MA_50,MA_200,log_return,RV_20,RV_50,symbol,time_idx,sentiment_B,sentiment
24712,24712,2023-12-07,203.060080,203.491916,201.764559,202.215591,1964500,0.0,0.0,12,...,192.021806,173.492307,178.567039,-0.000190,0.229097,0.315198,AMT,2190,NaN,-0.500000
216175,216175,2019-01-02,56.856821,58.581558,56.359406,58.554527,9361752,0.0,0.0,1,...,61.529831,65.518635,68.231328,0.016948,0.314171,0.310670,RTX,948,NaN,0.260521
133326,133326,2023-10-30,34.886519,35.368457,34.345564,35.102898,36656300,0.0,0.0,10,...,34.903732,34.998055,31.392888,0.004212,0.433922,0.374445,INTC,2163,NaN,0.368597
165775,165775,2016-09-07,70.225109,70.394831,69.497721,69.627037,4756500,0.0,0.0,9,...,70.170557,70.416977,67.329331,0.000000,0.110182,0.083259,MDT,365,NaN,0.000000
255727,255727,2018-04-19,38.319258,38.924703,38.303930,38.855728,10690900,0.0,0.0,4,...,38.884088,39.600156,41.073771,0.013903,0.144244,0.203503,USB,771,NaN,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137371,137371,2016-09-27,93.709891,94.366867,93.179564,94.366867,5349500,0.0,0.0,9,...,93.772433,94.266142,92.007045,0.012152,0.093410,0.083826,JNJ,379,NaN,0.160258
142551,142551,2023-06-22,136.327745,136.605562,133.108988,133.712509,10075200,0.0,0.0,6,...,133.890681,132.440900,124.871239,-0.019440,0.152327,0.242842,JPM,2073,NaN,-0.775803
105830,105830,2018-08-10,165.034716,165.369280,163.559199,164.545731,954300,0.0,0.0,8,...,168.657882,165.163027,177.086267,-0.003955,0.200990,0.156410,GD,850,NaN,0.981159
121148,121148,2017-04-03,193.786709,193.870969,190.054214,192.910461,3735600,0.0,0.0,4,...,201.125343,203.216596,167.154995,-0.003314,0.181633,0.197600,GS,508,NaN,0.056101


In [53]:
combined_df = combined_df.drop(columns=['sentiment_A', 'sentiment_B'])
combined_df = combined_df.dropna()
len(combined_df)

252847

## Make sure no NaN values left in dataset

In [54]:
null_counts = combined_df.isna().sum()

# Display the result
print(null_counts)
print(len(merged_df))
combined_df.sample(10)

Unnamed: 0      0
Date            0
Open            0
High            0
Low             0
Close           0
Volume          0
Dividends       0
Stock Splits    0
month           0
day             0
day_of_week     0
NASDAQ          0
SNP             0
DJI             0
RUT             0
VIX             0
XLK             0
XLE             0
XLF             0
XLV             0
RSI             0
MA_20           0
MA_50           0
MA_200          0
log_return      0
RV_20           0
RV_50           0
symbol          0
time_idx        0
sentiment       0
dtype: int64


NameError: name 'merged_df' is not defined

## Write Out Training Dataset

In [55]:
from datetime import datetime
combined_df.to_csv(f"{datetime.today()}_training.csv")